## Study of AMS-IX outage on the 13th of May 2015

In [ ]:
using Dates
using Glob
using LargeScaleAnalysis
using PyCall
using PyPlot

In [ ]:
# We use pandas since it has a more timeseries functions
# than DataFrames.jl and TimeSeries.jl.
pd = pyimport("pandas")
DateFormatter = pyimport("matplotlib.dates").DateFormatter
HourLocator = pyimport("matplotlib.dates").HourLocator;

In [ ]:
include("../../ParsimoniousMonitoring/notebooks/thesis.jl")
# tikzplotlib is not working very well with dates, so we use a mpl style.
plt.rc("figure", dpi = 120)
plt.style.use("classicthesis");

### Main

> The information we received from AMS-IX is that the primary cause of the event lasted for seven minutes and two seconds, from 2015-05-13T12:22:12+02:00 to 2015-05-13T12:29:14+02:00. This was the interval from a switch interface starting to generate looped traffic until the backbone interface of the switch was shut down, which immediately stopped the disruptive traffic from being propagated.

In [ ]:
EVENT_BEGIN = DateTime(2015, 5, 13, 10, 22, 12)
EVENT_END = DateTime(2015, 5, 13, 10, 29, 14);

In [ ]:
files = glob("*ndjson.model.json", "/blobs/datasets/thesis/2015-05-13.amsix/pairs");
models = map(x -> parsefile(DataSegmentationModel, x), files);
length(models)

In [ ]:
timestamps = map(x -> x.time, Iterators.flatten(changepoints.(models)));
datetimes = unix2datetime.(timestamps);

In [ ]:
df = pd.DataFrame(Dict(:timestamp => timestamps), index = datetimes)
df.head()

In [ ]:
fig, ax = subplots(figsize = (4.6, 2))
ax.plot(df.resample("360s").count())
ax.axvspan(EVENT_BEGIN, EVENT_END, alpha = 0.35, color = colorblindmap[3], lw = 0, label = "Incident AMS-IX")
ax.axvspan(
    DateTime(2015,5,13,15,45),
    DateTime(2015,5,13,16,0),
    color = colorblindmap[4],
    alpha = 0.35,
    label = "Incident DNS-A",
    lw = 0
)
ax.xaxis.set_major_formatter(DateFormatter("%d/%m\n%Hh%M"))
ax.xaxis.set_major_locator(HourLocator(byhour=[2,6,10,14,18,22]))
ax.set(xlabel = "UTC+0 (GMT)", ylabel = "Changements / 360s")
ax.grid()
ax.legend(loc = "upper left", fancybox = false)
plt.savefig("../figures/atlas-changefreq-ams-ix.pdf")